<a href="https://colab.research.google.com/github/armandoordonez/GenAI/blob/main/Intro_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install  packaging langchain langchain-community langchain-chroma

  Using cached langchain-0.3.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached langchain_core-0.3.7-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.1.129-py3-none-any.whl.metadata (13 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.5.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached chromadb-0.5.11-py3-none-any.whl.metadata (6.8 kB)
  Using cached fastapi-0.115.0-py3-none-any.whl.metadata (27 kB)
  Using cached chroma_hnswlib-0.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached uvicorn-0.31.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached posthog-3.6.6-py2.py3-none-a

In [4]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [9]:
!pip install langchain-openai # Install the missing langchain-openai package


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.8 MB/s eta 0:00:00


In [10]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

··········


In [11]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter



'Task decomposition is a technique that breaks down complex tasks into smaller, more manageable steps to enhance performance in models, particularly using Large Language Models (LLMs). This approach utilizes methods like Chain of Thought (CoT) and Tree of Thoughts that guide the model to think step-by-step and explore multiple reasoning possibilities. It can be executed through simple prompts, task-specific instructions, or human inputs to facilitate better task management and execution.'

In [13]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    #web_paths=("https://www.vaticannews.va/es/evangelio-de-hoy.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [14]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [15]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a complex task into smaller, manageable steps to facilitate easier execution. This can be accomplished through techniques such as Chain of Thought (CoT), where the model is prompted to think step by step, or through the Tree of Thoughts approach that explores multiple reasoning paths. Essentially, it enhances task performance by making intricate problems simpler to tackle.'

In [12]:
# cleanup
vectorstore.delete_collection()

In [ ]:
Ejempko de traza
# https://smith.langchain.com/public/1c6ca97e-445b-4d00-84b4-c7befcbc59fe/r